In [90]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import time

first_url = 'https://bigenc.ru/c/chiornaia-metallurgiia-97d762/'
base_url = 'https://bigenc.ru'


# хранение связей url
# title : {[[url], [annotation], [additional_title_list]] ... }
main_dict = dict()
second_iteration = dict()
# отработанные URL 
thrift_box = []


blacklist = ['http', 
            'versions',
            'references',
            'media',
            'about-project',
            'partners',
            'copyright-holders',
            'contacts',
            'tel:',
            'mailto:',
            'auth',
            'annotation',
            '/t/',
            '/l/',
            '/a/']

# Проверка наличия в полученном url слов из blacklist
def check_blacklist(link):
    for word in blacklist:
        if word in link or link == '/':
            return False
    return True

# Проверка наличия контента на странице (много пустых гиперссылок)
def check_url(url):
    child_dict = dict()
    response = requests.get(base_url+url)
    soup = BeautifulSoup(response.text, 'html.parser')
    if soup.find('h1', '-text-headline-1-small-x2 md:-text-headline-1-small tw-m-0') == None:
        title = soup.find('h1', 'bre-article-header-title').text
        annotation = get_annotation(response)
        child_dict[title] = {'annotation' : annotation,
                            'url' : base_url + url,
                            'additional_title' : []}
        return child_dict, title
    return None, None

# Парсим аннотацию, без XPATH обойтись не удалось, селекторы на страницах отличаются
def get_annotation(response):
    # загружаем HTML как объект lxml
    page = html.fromstring(response.text) 
    element = page.xpath('/html/body/div[1]/div[2]/main/div[1]/article/div[3]/div[2]/div[1]/div/div/section/section/p[1]')[0]
    annotation = element.text_content()
    return annotation

def main(url):
    work_dict = dict()
    if url in thrift_box:
        return None
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title = soup.find('h1', 'bre-article-header-title').text
    annotation = get_annotation(response)
    work_dict[title] = {'annotation' : annotation,
                        'url' : url,
                        'additional_title' : []}
    links = soup.find('body').find_all('a')
    for link in links:
        if len(work_dict[title]['additional_title']) >= 10:
            break
        additional_url = str(link.get('href'))
        if check_blacklist(additional_url):
            child_dict, child_title = check_url(additional_url)
            if child_dict != None:
                work_dict.update(child_dict)
                work_dict[title]['additional_title'].append(child_title)
    thrift_box.append(url)
    return work_dict

        
main_dict = main(first_url)
second_iteration = main_dict.copy()

for i in main_dict.keys():
    nw = (main(main_dict[i]['url']))
    if nw:
        second_iteration.update(nw)


In [97]:
print(len(main_dict))

11


In [100]:
import graphviz as gv

g = gv.Graph()


# Добавляем ребра для ключей в additional_title
for key, val in second_iteration.items():
    if 'additional_title' in val:
        for title in val['additional_title']:
            g.edge(key, title)


g.render('graph')

'graph.pdf'

In [96]:
with open('data.txt', 'w', encoding='utf-8') as f:
  for key, value in second_iteration.items():
    f.write(f'Title: {key}\n')
    f.write(f'Url: {value["url"]}\n') 
    f.write(f'Annotation: {value["annotation"]}\n\n')
